In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Grouping
* Grouping pixels into objects
    * "Perceptual organization"

## Why grouping?
* Pixel value is a property of sensor, not world
* Reasoning at object level might make things easy:
    * objects at consistent depth
    * objects can be recognized
    * objects move as one

## Regions ↔ Boundaries

# Edge Detection
## Why edges?
* Resilience to lighting and color
    * useful for recognition, matching patches across images
* Humans are sensitive to edges
* Convert a 2D image into a set of curves
    * Extracts salient features of the scene
    * More compact
* Cue to shape and geometry
    * useful for recognition, understanding 3D structure

## Edges
* Edges are curves in the image, across which the brightness changes "a lot"
* Corners / Junctions
* Different kinds of edges:
    * Depth discontinuities
    * Normal discontinuities
    * Discontinuities in "paint"
    * Shadows

## Characterizing edges
* Ad edge is a place of *rapid change* in the image intensity function

## Derivatives and Convolution
* Differentiation is *linear* $\frac{\partial (af(x) + bg(x)}{\partial x}$
* Differentiation is *shift-equivariant*
    * Derivative of shifted signal is shifted derivative
* Hence, differentiation can be represented as convolution

## Image derivatives
* How can we differentiate a *discrete* image $F[x,y]$?
    * Option 1: reconstruct a continuous image $f$, then compute the derivative
    * Option 2: take discrete derivative (finite difference)
    * $\frac{\partial f}{\partial x}[x,y] \approx F[x+1,y] - F[x,y]$
* Implement this as a linear filter
    * $\frac{\partial f}{\partial x}: H_x$
    * $\frac{\partial f}{\partial y}: H_y$

In [8]:
# Hx
hx = np.array([[0,0,0],[1,-1,0],[0,0,0]])
print(hx)

[[ 0  0  0]
 [ 1 -1  0]
 [ 0  0  0]]


In [9]:
# Hy
hy = np.array([[0,1,0],[0,-1,0],[0,0,0]])
print(hy)

[[ 0  1  0]
 [ 0 -1  0]
 [ 0  0  0]]


## Image gradient
* The *gradient* of an image: $\Delta f = [\frac{\partial f}{\partial x},\frac{\partial f}{\partial y}]$
    * The gradient points in the direction of most rapid increase in intensity
        * $\Delta f = [\frac{\partial f}{\partial x},0]$
        * $\Delta f = [0,\frac{\partial f}{\partial y}]$
        * $\Delta f = [\frac{\partial f}{\partial x},\frac{\partial f}{\partial y}]$
* The *edge strength* is given by the gradient magnitude:
    * $\| \Delta f \| = \sqrt{\big( \frac{\partial f}{\partial x} \big)^2 + \big( \frac{\partial f}{\partial y} \big)^2}$
* The gradient is given by:
    * $\theta = \tan^{-1} \big( \frac{\partial f}{\partial x} / \frac{\partial f}{\partial y} \big)$
        * How does this relate to the directino of the edge?

### Gradient in $x$

### Gradient in $y$

## Effect of noise
* Input
* Gradient magnitude

### Noisy input image
* $f(x)$
* $\frac{d}{dx}f(x)$
* Hard to find edge


* Noise is high frequency
* Differentiation accentuates noise
    * $\frac{d \sin wx}{dx} = w cos wx$

### Solution : Smooth first
* To find edges, look for peaks in $\frac{d}{dx}(f*h)$

## Associative property of convolution
* Differentiation is a convolution
* Convolution is associative:
    * $\frac{d}{dx}(f*h) = f * \frac{d}{dx}h$

## 2D edge detection filters
* Gaussian filter
    * $h_\sigma (u,v) = \frac{1}{2\pi \sigma ^2}e^{-\frac{u^2+v^2}{2\sigma^2}}$
* Derivative of the Gaussian filter
    * $\frac{\partial}{\partial x}h_\sigma (u,v)$

### Derivative of the Gaussian filter
* $x$-direction
* $y$-direction

### Anisotropic Gaussian
* Should smoothing along the edge be the same as smoothing across it?
* Smooth more along the edge than across it
* Use a lower standard deviation along one direction
* $G(x,y) = e^{-\frac{x^2}{2\sigma^2_x}-\frac{y^2}{2\sigma^2_y}}$
* Can be applied to derivative

### The Sobel filter
* A variant of the Anisotropic Gaussian
* Smooth *only* along the edge
* $x$ derivative filter

## Oriented Gaussian Derivatives
* Instead of just derivative along $x$ and $y$, we can also rotate filter to get derivatives along different direction

### Which way does the gradient point?

## Gradient Magnitude & Gradient Orientation

## From gradients to boundaries?
* Can we threshold the gradient magnitude to get object boundaries?
* Three different thresholds:
    * False positives
    * Thick boundaries
    * Gaps

### Thick boundaries
* Want to find *peaks* in the gradient magnitude
* As one moves across edge, gradient magnitude rises and then falls
* Find the peak
    * Image → Intensity function → First derivative

## Non-maximum suppression for each orientation
* Let $g(q)$ be the image gradient at pixel $q$
    * A vector
* Take a step forwards and backwards along gradient
    * $r = q + \frac{g(q)}{\| g(q) \|}$
    * $p = q - \frac{g(q)}{\| g(q) \|}$
* Find gradient magnitude at $p$ and $r$
    * Interpolate into gradient magnitude
* If magnitude at $q$ is greater at $p$ and $r$, **$q$ is a local peak of gradient magnitude**

### Non-maximum suppresion (Before & After)

### Non-maximum suppresion (With various thresholds)

## Hysteresis thresholding
* Start with a high threshold
* Then grow high threshold edges by looking for neighbors that clear a lower threshold
    * White: Clears high threshold
    * Gray: Clears low threshold but not high threshold
        * Mark pixels that clear high threshold as boundary
        * If there are pixels that clear low threshold and are connected to a boundary, mark them as boundary too

### High threshold | Low threshold | Hysteresis threshold

## Canny edge detector
1. Filter image with $x,y$ derivatives of Gaussian
2. Find magnitude and orientation of gradient
3. Non-maximum suppression:
    * This multi-pixel wide "ridges" down to single pixel width
4. Thresholding and linking (hysteresis):
    * Define two thresholds: low and high
    * Use the high threshold to start edge curves and the low threshold to continue them

### Texture & Low-contrast edges